# STEP #0: PROBLEM STATEMENT  

- Transfer learning is a machine learning technique in which intelligence (i.e.: weights) from a base artificial neural network is being transferred to a new network as a starting point to perform a specific task. 
- Transfer learning is a key skill to acquire as a data scientist since it can dramatically reduce the computational time required if the network is being trained from scratch (i.e.: starting from purely random weights). 
- In this project, we are going to use a trained ANN known as MobileNet that come readily trained/available from TF 2.0 Hub
- We are going to use TensorFlow Flowers dataset 
- This code is based on TensorFlow 2.0 Hub documentation: 
https://www.tensorflow.org/beta/tutorials/images/hub_with_keras


# STEP #1: IMPORT LIBRARIES

In [ ]:
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random 


In [ ]:
!pip install tensorflow_hub
import tensorflow_hub as hub


# STEP #2: IMPORT MOBILENET WITH PRE-TRAINED WEIGHTS FROM TF 2.0 HUB

In [ ]:
Trained_MobileNet_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}

In [ ]:
Trained_MobileNet = tf.keras.Sequential([
    hub.KerasLayer(Trained_MobileNet_url, input_shape=(224,224,3))])

# STEP #3: EVALUATE PRE-TRAINED MOBILENET (JUST TO BE SURE :)!)

- Check this out: http://www.image-net.org/synset?wnid=n02835271
- Search for Bicycle
- https://lear.inrialpes.fr/people/gordo/ImageNetResults/sift1000_accuracy_withPrior/results_sift_0253.htm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
Sample_Image= tf.keras.preprocessing.image.load_img(r'/content/drive/My Drive/Colab Notebooks/TF 2.0 Advanced/Transfer Learning Data/watch.jpg', target_size = (224, 224))

In [ ]:
plt.imshow(Sample_Image)

In [ ]:
Sample_Image = np.array(Sample_Image)/255.0


In [ ]:
predicted_class = Trained_MobileNet.predict(np.expand_dims(Sample_Image, axis = 0))

In [ ]:
predicted_class

In [ ]:
predicted_class.shape

In [ ]:
predicted_class = np.argmax(predicted_class)
predicted_class

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
plt.imshow(Sample_Image)
predicted_class_name = imagenet_labels[predicted_class]
plt.title("Predicted Class is: " + predicted_class_name.title())

# STEP #4: LET'S TEST THE MODEL ON A BATCH OF IMAGES

In [ ]:
# Specify path of the flowers dataset
flowers_data_url = tf.keras.utils.get_file(
  'flower_photos','https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
   untar=True)

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
flowers_data = image_generator.flow_from_directory(str(flowers_data_url), target_size=(224,224), batch_size = 64, shuffle = True)

In [ ]:
for flowers_data_input_batch, flowers_data_label_batch in flowers_data:
  print("Image batch shape: ", flowers_data_input_batch.shape)
  print("Label batch shape: ", flowers_data_label_batch.shape)
  break

In [ ]:
predictions_batch = Trained_MobileNet.predict(flowers_data_input_batch)
predictions_batch.shape

In [ ]:
predicted_class_names = imagenet_labels[np.argmax(predictions_batch, axis=-1)]
predicted_class_names

In [ ]:
plt.figure(figsize=(15,15))

for n in range(64):
  plt.subplot(8,8,n+1)
  plt.imshow(flowers_data_input_batch[n])
  plt.title(predicted_class_names[n])
  plt.axis('off')

# Network performance is poor because it wasn't even trained to clasify these types of flowers!

# STEP #5: APPLY TRANSFER LEARNING AND RETRAIN THE MODEL

You can download the MobileNet without the classification head:
https://tfhub.dev/s?module-type=image-feature-vector&q=tf2


In [ ]:
MobileNet_feature_extractor_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2" #@param {type:"string"}

In [ ]:
MobileNet_feature_extractor_layer = hub.KerasLayer(MobileNet_feature_extractor_url, input_shape=(224, 224, 3))

In [ ]:
feature_batch = MobileNet_feature_extractor_layer(flowers_data_input_batch)
print(feature_batch.shape)

In [ ]:
MobileNet_feature_extractor_layer.trainable = False

In [ ]:
flowers_data.num_classes

In [ ]:
# Build a model with two pieces:
#    (1)  MobileNet Feature Extractor 
#    (2)  Dense Network (classifier) added at the end 

model = tf.keras.Sequential([
  MobileNet_feature_extractor_layer,
  tf.keras.layers.Dense(flowers_data.num_classes, activation='softmax')
])

model.summary()

In [ ]:
predictions = model(flowers_data_input_batch)

In [ ]:
predictions.shape

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(flowers_data, epochs=50)

In [ ]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

# STEP #6: EVALUATE THE MODEL

In [ ]:
class_names = sorted(flowers_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

In [ ]:
predicted_batch = model.predict(flowers_data_input_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

In [ ]:
label_id = np.argmax(flowers_data_output_batch, axis=-1)

In [ ]:
label_id

In [ ]:
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)

for n in range(64):
  plt.subplot(8,8,n+1)
  plt.imshow(flowers_data_input_batch[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')


# GREAT JOB!